In [1]:
!pip install icd10-cm bgen_reader pydantic

     |████████████████████████████████| 675kB 9.0MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 10.1MB 17.8MB/s 
     |████████████████████████████████| 768kB 41.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 286kB 38.9MB/s 
     |████████████████████████████████| 532kB 46.9MB/s 
     |████████████████████████████████| 112kB 46.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfu

In [2]:
from bgen_reader import read_bgen, open_bgen
from pydantic import BaseModel
import numpy as np
import dask.dataframe as dd
from dask.delayed import Delayed
from typing import List, Union


In [3]:
#utils functions

def get_geno_one_snp(row,  high_lim=0.9, low_lim=0.3, NA_val = np.nan):
    geno_1, geno_2, geno_3 = row
    homo_ref_cond = (geno_1 >=high_lim) & (geno_2 < low_lim) & (geno_3 < low_lim)
    het_cond = (geno_2 >= high_lim) & (geno_1 < low_lim) & (geno_3 < low_lim)
    homo_alt_cond = (geno_3 >= high_lim) & (geno_1 < low_lim) & (geno_2 < low_lim)
    geno_df =  np.select([homo_ref_cond, het_cond, homo_alt_cond],
             [0., 1., 2.],
             default = NA_val)
    return geno_df


In [ ]:
class BgenFileObject(BaseModel):
    variants: dd.DataFrame
    samples: pd.Series
    genotype: List[Delayed]
    bgen_reader_obj: open_bgen
    
    class Config:
        arbitrary_types_allowed = True
    
    def __repr__(self):
        return str(self.__class__) + f" {self.samples.shape[0]} samples"
    
    def get_variant_index(self,rsids=None):
        variant_index = np.argwhere(np.isin(self.bgen_reader_obj.rsids, rsids)).reshape(-1,) if rsids is not None else None
        return variant_index
    
    def get_sample_index(self, sample_ids=None):
        sample_index = np.argwhere(np.isin(self.samples.values, sample_ids)).reshape(-1,) if sample_ids is not None else None
        return sample_index
    
    def get_probs(self, sample_ids=None, rsids=None):
        variant_index = self.get_variant_index(rsids)
        sample_index = self.get_sample_index(sample_ids)
        return self.bgen_reader_obj.read((sample_index, variant_index))
    
    
    def get_geno_each_sample(self, probs, prob_to_geno_func:Union["max", "stringent"]= "stringent", high_lim=0.9, low_lim=0.3, NA_val=np.nan):
        if prob_to_geno_func == "max":
            geno = np.nanargmax(probs, axis=2).astype(float)
            
        elif prob_to_geno_func == "stringent":
            geno = np.apply_along_axis(get_geno_one_snp, axis=2, arr=probs, high_lim=high_lim, low_lim=low_lim, NA_val=NA_val)
        
        return geno
            
        
    def get_allele_ids(self, rsids = None, variant_index = None):
        if variant_index is None:
            variant_index = self.get_variant_index(rsids)
        df = pd.DataFrame([allele_str.split(",") for allele_str in self.bgen_reader_obj.allele_ids[variant_index]], columns = ["allele_1", "allele_2"])
        
        if rsids is not None:
            df.index = rsids
        return df
    
    def get_variant_combinations(self, rsids = None, variant_index = None):
        if variant_index is None:
            variant_index = np.argwhere(np.isin(self.bgen_reader_obj.rsids, rsids)).reshape(-1,) if rsids is not None else None
        geno_df = self.get_allele_ids(rsids, variant_index)
        geno_df = get_possible_geno_combinations(geno_df, "allele_1", "allele_2")
        return geno_df

In [ ]:
#the first time reading the file, the metafile will be created so no need to put it as argument.
#Please modify the path of the file as appropriate, your path might be different than mine
filepath = "/content/drive/MyDrive/6.874 project/Data/ukb_imp_chr21_v3.bgen"
samples_filepath="/content/drive/MyDrive/6.874 project/Data/ukb45624_imp_chr21_v3_s487275.sample"

test = BgenFileObject(**read_bgen(filepath=filepath, samples_filepath=samples_filepath), bgen_reader_obj = open_bgen(filepath=filepath, samples_filepath=samples_filepath))


Sample IDs are read from /content/drive/MyDrive/6.874 project/Data/ukb45624_imp_chr21_v3_s487275.sample.


Mapping genotypes: 100%|██████████| 1261158/1261158 [01:33<00:00, 13546.64it/s]



Sample IDs are read from '/content/drive/MyDrive/6.874 project/Data/ukb45624_imp_chr21_v3_s487275.sample''.




In [ ]:
test.samples

0         5542886
1         5137974
2         3758348
3         1391800
4         3165331
           ...   
487404    5512806
487405    5548469
487406    2956972
487407    5229561
487408    3665101
Name: id, Length: 487409, dtype: object

In [ ]:
test.bgen_reader_obj.rsids

memmap(['rs559462325', 'rs181691356', 'rs548263598', ..., 'rs555877612',
        'rs574115117', 'rs541185110'], dtype='<U115')

In [ ]:
test_probs = test.get_probs(sample_ids = ['5542886', '5137974', '3758348', '1391800'], rsids = ['rs559462325', 'rs181691356', 'rs548263598', 'rs555877612', 'rs574115117', 'rs541185110'])
test_probs

reading -- time=0:00:00.10, thread 1 of 2, part 3 of 3


array([[[1.        , 0.        , 0.        ],
        [0.99215686, 0.00784314, 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [0.99215686, 0.00784314, 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [1.        , 0.     

In [ ]:
test_probs.shape

(4, 6, 3)

In [ ]:
test.get_geno_each_sample(test_probs)

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
test.get_geno_each_sample(test_probs).shape

(4, 6)

In [ ]:
np.identity(3)[test.get_geno_each_sample(test_probs).astype(int)]

array([[[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]])

In [ ]:
np.identity(3)[test.get_geno_each_sample(test_probs).astype(int)].shape

(4, 6, 3)

In [ ]:
# output: binary matrix of (n samples, p phenotypes)

In [ ]:
import pandas as pd

In [ ]:
pheno = pd.read_csv("/content/drive/MyDrive/6.874 project/Data/ukb42682_icd10.txt", sep="\t")
pheno

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


eid 41202-0.0 41202-0.1  ... 41204-0.181 41204-0.182 41204-0.183
NaN 6026424   NaN       NaN       NaN  ...         NaN         NaN         NaN
    1000011  E831      H251      I249  ...         NaN         NaN         NaN
    1000026  M179     S8250       NaN  ...         NaN         NaN         NaN
    1000032  H269      H449      R073  ...         NaN         NaN         NaN
    1000044  H333       NaN       NaN  ...         NaN         NaN         NaN
...           ...       ...       ...  ...         ...         ...         ...
    6026376   NaN       NaN       NaN  ...         NaN         NaN         NaN
    6026389   NaN       NaN       NaN  ...         NaN         NaN         NaN
    6026393  D361      M161       NaN  ...         NaN         NaN         NaN
    6026407  K047      N939       NaN  ...         NaN         NaN         NaN
    6026419  D223      O039       NaN  ...         NaN         NaN         NaN

[502505 rows x 251 columns]

In [ ]:
import icd10

code = icd10.find("E.83.1")
code.description

'Disorders of iron metabolism'